## Semantic chunkings
pros:
- Higher Relevance & Accuracy
- Reduced Hallucination Risk

cons:
- Complexity & Computational Overhead
- Inconsistent Chunk Sizes

### When to Use Semantic Chunks?
- Ideal For: Narrative text (books, articles, documentation), where ideas flow in paragraphs. For Q&A on specific, localized facts or explanations.

- Good For: High-stakes applications where retrieval accuracy and answer quality are paramount, and preprocessing cost is acceptable.

- Often Used In Combination with other strategies (e.g., semantic chunking with a small overlap to mitigate cross-chunk issues).


In [1]:
# simple and base
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [25]:
text = """هوش مصنوعی بسیاری از صنایع را متحول کرده است.

در حوزه مراقبت‌های بهداشتی، الگوریتم‌های هوش مصنوعی اکنون می‌توانند بیماری‌ها را از تصاویر پزشکی با دقتی فراتر از متخصصان انسانی تشخیص دهند.

این نشان‌دهنده یک پیشرفت بزرگ در پزشکی تشخیصی است.

بخش مالی از هوش مصنوعی برای تشخیص کلاهبرداری و معاملات الگوریتمی استفاده می‌کند.

مدل‌های یادگیری ماشین، الگوهای موجود در داده‌های تراکنش را برای شناسایی فعالیت‌های مشکوک در زمان واقعی تجزیه و تحلیل می‌کنند.

علم آب و هوا از طریق بهبود پیش‌بینی آب و هوا و مدل‌سازی آب و هوا از هوش مصنوعی بهره می‌برد.

شبکه‌های عصبی مقادیر زیادی از داده‌های ماهواره‌ای را برای پیش‌بینی رویدادهای شدید آب و هوایی پردازش می‌کنند."""

sentences = [s.strip() for s in text.split("\n") if s.strip()]
len(sentences)


7

In [26]:
embeddings = model.encode(sentences)


In [27]:
embeddings[0][:2]

array([ 0.05292744, -0.28391474], dtype=float32)

In [31]:
chunks =[]
current_chunks =[sentences[0]]
for i  in range (1, len(sentences)):
    sim = cosine_similarity([embeddings[i-1]],[embeddings[i]])[0][0]
    
    if sim > 0.5:
        current_chunks.append(sentences[i])
    else:
        chunks.append(" ".join(current_chunks))
        current_chunks = [sentences[i]]
chunks.append(" ".join(current_chunks))

In [32]:
for i, c in enumerate(chunks):
    print(f"no: {i} , \n chunks: {c}")

no: 0 , 
 chunks: هوش مصنوعی بسیاری از صنایع را متحول کرده است. در حوزه مراقبت‌های بهداشتی، الگوریتم‌های هوش مصنوعی اکنون می‌توانند بیماری‌ها را از تصاویر پزشکی با دقتی فراتر از متخصصان انسانی تشخیص دهند.
no: 1 , 
 chunks: این نشان‌دهنده یک پیشرفت بزرگ در پزشکی تشخیصی است.
no: 2 , 
 chunks: بخش مالی از هوش مصنوعی برای تشخیص کلاهبرداری و معاملات الگوریتمی استفاده می‌کند.
no: 3 , 
 chunks: مدل‌های یادگیری ماشین، الگوهای موجود در داده‌های تراکنش را برای شناسایی فعالیت‌های مشکوک در زمان واقعی تجزیه و تحلیل می‌کنند.
no: 4 , 
 chunks: علم آب و هوا از طریق بهبود پیش‌بینی آب و هوا و مدل‌سازی آب و هوا از هوش مصنوعی بهره می‌برد. شبکه‌های عصبی مقادیر زیادی از داده‌های ماهواره‌ای را برای پیش‌بینی رویدادهای شدید آب و هوایی پردازش می‌کنند.


In [33]:
len(chunks)

5

In [49]:
from semantic_text_splitter import TextSplitter

splitter = TextSplitter((100,500))
chunks2= splitter.chunks(text)
len(chunks2)

4

In [50]:
for i, c in enumerate(chunks2):
    print(f"no: {i} , \n chunks: {c}")

no: 0 , 
 chunks: هوش مصنوعی بسیاری از صنایع را متحول کرده است.

در حوزه مراقبت‌های بهداشتی، الگوریتم‌های هوش مصنوعی اکنون می‌توانند بیماری‌ها را از تصاویر پزشکی با دقتی فراتر از متخصصان انسانی تشخیص دهند.
no: 1 , 
 chunks: این نشان‌دهنده یک پیشرفت بزرگ در پزشکی تشخیصی است.

بخش مالی از هوش مصنوعی برای تشخیص کلاهبرداری و معاملات الگوریتمی استفاده می‌کند.
no: 2 , 
 chunks: مدل‌های یادگیری ماشین، الگوهای موجود در داده‌های تراکنش را برای شناسایی فعالیت‌های مشکوک در زمان واقعی تجزیه و تحلیل می‌کنند.
no: 3 , 
 chunks: علم آب و هوا از طریق بهبود پیش‌بینی آب و هوا و مدل‌سازی آب و هوا از هوش مصنوعی بهره می‌برد.

شبکه‌های عصبی مقادیر زیادی از داده‌های ماهواره‌ای را برای پیش‌بینی رویدادهای شدید آب و هوایی پردازش می‌کنند.


In [ ]:
# another way 
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings

loader = TextLoader("تال چیست.txt",encoding="utf-8")
docs = loader.load()

embeddings_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

chunker = SemanticChunker(
    embeddings=embeddings_model,
    breakpoint_threshold_amount=50,  # Adjust this (80-95 typical)
    min_chunk_size=50  #  min chunk size in characters 
)

chunks3 = chunker.split_text(text) # has document splitter (.split_documents)

for i, c in enumerate(chunks3):
    print(f"no: {i} , \n chunks: {c}")

no: 0 , 
 chunks: هوش مصنوعی بسیاری از صنایع را متحول کرده است. در حوزه مراقبت‌های بهداشتی، الگوریتم‌های هوش مصنوعی اکنون می‌توانند بیماری‌ها را از تصاویر پزشکی با دقتی فراتر از متخصصان انسانی تشخیص دهند. این نشان‌دهنده یک پیشرفت بزرگ در پزشکی تشخیصی است. بخش مالی از هوش مصنوعی برای تشخیص کلاهبرداری و معاملات الگوریتمی استفاده می‌کند.
no: 1 , 
 chunks: مدل‌های یادگیری ماشین، الگوهای موجود در داده‌های تراکنش را برای شناسایی فعالیت‌های مشکوک در زمان واقعی تجزیه و تحلیل می‌کنند.
no: 2 , 
 chunks: علم آب و هوا از طریق بهبود پیش‌بینی آب و هوا و مدل‌سازی آب و هوا از هوش مصنوعی بهره می‌برد.
no: 3 , 
 chunks: شبکه‌های عصبی مقادیر زیادی از داده‌های ماهواره‌ای را برای پیش‌بینی رویدادهای شدید آب و هوایی پردازش می‌کنند.
